In [260]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from xgboost import XGBClassifier, plot_importance
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [223]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [224]:
df_train.head()

id  Gender  Age  Driving_License  Region_Code  Previously_Insured  \
0   1    Male   44                1         28.0                   0   
1   2    Male   76                1          3.0                   0   
2   3    Male   47                1         28.0                   0   
3   4    Male   21                1         11.0                   1   
4   5  Female   29                1         41.0                   1   

  Vehicle_Age Vehicle_Damage  Annual_Premium  Policy_Sales_Channel  Vintage  \
0   > 2 Years            Yes         40454.0                  26.0      217   
1    1-2 Year             No         33536.0                  26.0      183   
2   > 2 Years            Yes         38294.0                  26.0       27   
3    < 1 Year             No         28619.0                 152.0      203   
4    < 1 Year             No         27496.0                 152.0       39   

   Response  
0         1  
1         0  
2         1  
3         0  
4         0

In [225]:
df_train.duplicated('Policy_Sales_Channel', keep = False)

0         True
1         True
2         True
3         True
4         True
          ... 
381104    True
381105    True
381106    True
381107    True
381108    True
Length: 381109, dtype: bool

In [226]:
df_train.drop_duplicates('Policy_Sales_Channel')

id  Gender  Age  Driving_License  Region_Code  Previously_Insured  \
0            1    Male   44                1         28.0                   0   
3            4    Male   21                1         11.0                   1   
5            6  Female   24                1         33.0                   0   
10          11  Female   47                1         35.0                   0   
12          13  Female   41                1         15.0                   1   
...        ...     ...  ...              ...          ...                 ...   
262746  262747  Female   28                1         36.0                   1   
272336  272337    Male   39                1         28.0                   0   
279714  279715    Male   67                1         28.0                   1   
324173  324174    Male   74                1         17.0                   0   
364157  364158  Female   40                1          0.0                   1   

       Vehicle_Age Vehicle_Damage  Annual_Premium  Policy_Sales_Channel  \
0        > 2 Years            Yes         40454.0                  26.0   
3         < 1 Year             No         28619.0                 152.0   
5         < 1 Year            Yes          2630.0                 160.0   
10        1-2 Year            Yes         47576.0                 124.0   
12        1-2 Year             No         31409.0                  14.0   
...            ...            ...             ...                   ...   
262746    < 1 Year             No         37645.0                   6.0   
272336    1-2 Year            Yes         40220.0                  50.0   
279714    1-2 Year             No         33236.0                 144.0   
324173    1-2 Year            Yes          2630.0                 143.0   
364157    1-2 Year             No          2630.0                  41.0   

        Vintage  Response  
0           217         1  
3           203         0  
5           176         0  
10           46         1  
12          221         0  
...         ...       ...  
262746      265         0  
272336       92         0  
279714      290         0  
324173       15         0  
364157      200         0  

[155 rows x 12 columns]

In [227]:
df_train['Policy_Sales_Channel'].value_counts()

152.0    134784
26.0      79700
124.0     73995
160.0     21779
156.0     10661
          ...  
149.0         1
43.0          1
144.0         1
143.0         1
41.0          1
Name: Policy_Sales_Channel, Length: 155, dtype: int64

# 전처리

In [228]:
# 필요없는 컬럼 제거
df_train.drop(columns = ['id'], inplace = True) 

In [229]:
#면허증 없는 고객들 행 제거
df_train.drop(df_train[(df_train['Driving_License'] == 0)].index, inplace=True)

In [230]:
#면허증 없는 고객들 행 제거했으니 면허증 컬럼엔 모두 동일한 값만 남기 때문에 해당 컬럼 제거
df_train.drop(columns = ['Driving_License'], inplace = True) 

In [231]:
# Policy_Sales_Channel 채널 value_counts로수정하기(랭크매기는 효과)
# df_train['Policy_Sales_Channel_count'] = df_train['Policy_Sales_Channel'].map(lambda x: df_train['Policy_Sales_Channel'].value_counts()[x])

# 새로운 칼럼 생성하기
df_train['Policy_Sales_Channel_AVG_Prem'] = df_train.groupby('Policy_Sales_Channel')['Annual_Premium'].transform('mean') # 판매채널 별 평균 보험료
df_train['Region_Code_AVG_Prem'] = df_train.groupby('Region_Code')['Annual_Premium'].transform('mean') # 지역별 평균 보험료


In [232]:
# Gender 컬럼 숫자화
df_train['Gender'] = df_train['Gender'].map( {'Female': 0, 'Male': 1} ).astype(int)

In [233]:
# 원핫 인코딩
df_train=pd.get_dummies(df_train,drop_first=True)

In [234]:
df_train=df_train.rename(columns={"Vehicle_Age_< 1 Year": "Vehicle_Age_lt_1_Year", "Vehicle_Age_> 2 Years": "Vehicle_Age_gt_2_Years"})
df_train['Vehicle_Age_lt_1_Year']=df_train['Vehicle_Age_lt_1_Year'].astype('int')
df_train['Vehicle_Age_gt_2_Years']=df_train['Vehicle_Age_gt_2_Years'].astype('int')
df_train['Vehicle_Damage_Yes']=df_train['Vehicle_Damage_Yes'].astype('int')

In [235]:
df_train["premium_age_ratio"] = df_train["Annual_Premium"]/df_train["Age"]
df_train["premium_vintage_ratio"] = df_train["Annual_Premium"]/df_train["Vintage"]

In [236]:
import numpy as np
arr = df_train['Policy_Sales_Channel'].to_numpy()
values, counts = np.unique(arr, return_counts=True)
count_dict = dict(zip(values, counts))

df_train['Policy_Sales_Channel_count'] = df_train['Policy_Sales_Channel'].map(count_dict)
df_train['Region_Code_count'] = df_train['Region_Code'].map(count_dict)

In [239]:
# 전처리 데이터에 있는 null값 KNN으로 채우기

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_train['Region_Code_count'] = imputer.fit_transform(df_train[['Region_Code_count']])

In [240]:
num_feat = ['Age','premium_age_ratio','premium_vintage_ratio', 'Policy_Sales_Channel_AVG_Prem', 'Region_Code_AVG_Prem', 'Policy_Sales_Channel_count', 'Region_Code_count']
ss = StandardScaler()
df_train[num_feat] = ss.fit_transform(df_train[num_feat])

In [241]:
mm = MinMaxScaler()
df_train[['Annual_Premium']] = mm.fit_transform(df_train[['Annual_Premium']])

In [242]:
cat_feat = ['Gender', 'Previously_Insured', 'Vehicle_Age_lt_1_Year','Vehicle_Age_gt_2_Years','Vehicle_Damage_Yes']
for column in cat_feat:
    df_train[column] = df_train[column].astype('str')

In [243]:
df_train.drop(columns = ['Region_Code', 'Policy_Sales_Channel'], inplace = True) 

In [244]:
df_train = df_train[['Gender', 'Age', 'Previously_Insured', 'Annual_Premium', 'Policy_Sales_Channel_AVG_Prem', 'Region_Code_AVG_Prem', 'Vehicle_Age_lt_1_Year', 'Vehicle_Age_gt_2_Years', 'Vehicle_Damage_Yes', 'premium_age_ratio', 'premium_vintage_ratio', 'Policy_Sales_Channel_count', 'Region_Code_count', 'Response']]

In [245]:
df_train

Gender       Age Previously_Insured  Annual_Premium  \
0           1  0.338348                  0        0.070366   
1           1  2.406723                  0        0.057496   
2           1  0.532258                  0        0.066347   
3           1 -1.148298                  1        0.048348   
4           0 -0.631204                  1        0.046259   
...       ...       ...                ...             ...   
381104      1  2.277450                  1        0.051234   
381105      1 -0.566567                  1        0.069551   
381106      1 -1.148298                  1        0.060439   
381107      0  1.889629                  0        0.078110   
381108      1  0.467621                  0        0.072827   

        Policy_Sales_Channel_AVG_Prem  Region_Code_AVG_Prem  \
0                            0.593621              1.098084   
1                            0.593621             -0.818229   
2                            0.593621              1.098084   
3                            0.064258             -0.399041   
4                            0.064258              0.052733   
...                               ...                   ...   
381104                       0.593621             -0.183907   
381105                       0.064258             -0.278466   
381106                      -0.955846             -0.687736   
381107                       0.221017             -0.645383   
381108                       0.593621             -0.156214   

       Vehicle_Age_lt_1_Year Vehicle_Age_gt_2_Years Vehicle_Damage_Yes  \
0                          0                      1                  1   
1                          0                      0                  0   
2                          0                      1                  1   
3                          1                      0                  0   
4                          1                      0                  0   
...                      ...                    ...                ...   
381104                     0                      0                  0   
381105                     1                      0                  0   
381106                     1                      0                  0   
381107                     0                      1                  1   
381108                     0                      0                  0   

        premium_age_ratio  premium_vintage_ratio  Policy_Sales_Channel_count  \
0                0.021048              -0.323118                   -0.030742   
1               -0.774199              -0.328891                   -0.030742   
2               -0.152993               1.922445                   -0.030742   
3                0.758507              -0.405957                    1.122137   
4                0.068830               0.622234                    1.122137   
...                   ...                    ...                         ...   
381104          -0.830016              -0.037988                   -0.030742   
381105           0.710370              -0.106118                    1.122137   
381106           1.273225              -0.265332                   -1.225801   
381107          -0.416831               0.436130                   -0.144773   
381108           0.002398              -0.341620                   -0.030742   

        Region_Code_count  Response  
0               -0.141136         1  
1               -0.061822         0  
2               -0.141136         1  
3                0.042548         0  
4               -0.141444         0  
...                   ...       ...  
381104          12.046731         0  
381105          -0.118233         0  
381106           0.073444         0  
381107          -0.046912         0  
381108          -0.012173         0  

[380297 rows x 14 columns]

In [246]:
# df_train.to_csv('train_prep.csv')

In [249]:
training_data, testing_data = df_train.drop('Response', axis=1),df_train['Response']

In [250]:
x_train, x_test, y_train, y_test = train_test_split(training_data,testing_data, test_size=0.2, random_state=42)

In [251]:
x_train

Gender       Age Previously_Insured  Annual_Premium  \
299688      1 -0.825114                  1        0.088015   
168936      1 -1.148298                  0        0.000000   
231982      0 -0.889751                  0        0.069659   
113352      1  0.079801                  1        0.121278   
224811      0 -0.178746                  0        0.122238   
...       ...       ...                ...             ...   
259732      1  0.015164                  0        0.034770   
366617      1 -0.889751                  1        0.035486   
132207      0  0.144437                  0        0.000000   
147176      0  2.277450                  0        0.000000   
122201      1 -0.437293                  0        0.080090   

        Policy_Sales_Channel_AVG_Prem  Region_Code_AVG_Prem  \
299688                       0.064258              0.126764   
168936                       0.064258             -0.561924   
231982                       0.064258             -0.125680   
113352                       0.221017              1.098084   
224811                       1.221558              1.098084   
...                               ...                   ...   
259732                       0.221017              0.800305   
366617                       0.064258             -0.716737   
132207                       0.593621             -0.298411   
147176                      -4.854055             -3.776199   
122201                      -3.674893              1.098084   

       Vehicle_Age_lt_1_Year Vehicle_Age_gt_2_Years Vehicle_Damage_Yes  \
299688                     1                      0                  0   
168936                     1                      0                  1   
231982                     1                      0                  1   
113352                     0                      0                  0   
224811                     0                      0                  1   
...                      ...                    ...                ...   
259732                     0                      0                  1   
366617                     1                      0                  0   
132207                     0                      0                  0   
147176                     0                      0                  1   
122201                     1                      0                  1   

        premium_age_ratio  premium_vintage_ratio  Policy_Sales_Channel_count  \
299688           1.686564              -0.212271                    1.122137   
168936          -1.299808              -0.635236                    1.122137   
231982           1.157923               0.925103                    1.122137   
113352           1.311876              -0.098427                   -0.144773   
224811           1.649046               0.595341                   -1.473353   
...                   ...                    ...                         ...   
259732          -0.598893              -0.476102                   -0.144773   
366617          -0.064122               0.128367                    1.122137   
132207          -1.401415              -0.643847                   -0.030742   
147176          -1.448992              -0.615480                   -1.659962   
122201           0.866152              -0.224678                   -1.456855   

        Region_Code_count  
299688          -0.139138  
168936          -0.139292  
231982          -0.005717  
113352          -0.141136  
224811          -0.141136  
...                   ...  
259732           0.090659  
366617          -0.134527  
132207          -0.115928  
147176          -0.138523  
122201          -0.141136  

[304237 rows x 13 columns]

In [252]:
x_test

Gender       Age Previously_Insured  Annual_Premium  \
347722      1  1.437172                  1        0.050758   
216616      0 -1.083661                  1        0.036902   
136543      0 -1.083661                  1        0.069275   
320938      1  1.178625                  0        0.065047   
361802      0 -0.954387                  0        0.075571   
...       ...       ...                ...             ...   
44283       1  0.015164                  1        0.050246   
139726      0  0.079801                  0        0.047664   
178713      1 -1.019024                  1        0.000000   
181347      1  0.015164                  0        0.054201   
347606      1  0.079801                  0        0.051729   

        Policy_Sales_Channel_AVG_Prem  Region_Code_AVG_Prem  \
347722                       0.593621             -0.125680   
216616                      -0.955846             -0.454577   
136543                      -0.955846              0.800305   
320938                       0.221017             -0.505397   
361802                       0.064258             -0.125680   
...                               ...                   ...   
44283                        1.221558             -0.687736   
139726                       0.221017              0.800305   
178713                      -0.239466             -0.792052   
181347                       0.221017             -0.541541   
347606                       0.221017              1.098084   

       Vehicle_Age_lt_1_Year Vehicle_Age_gt_2_Years Vehicle_Damage_Yes  \
347722                     0                      0                  1   
216616                     1                      0                  0   
136543                     1                      0                  0   
320938                     0                      1                  1   
361802                     1                      0                  1   
...                      ...                    ...                ...   
44283                      0                      0                  0   
139726                     0                      0                  1   
178713                     1                      0                  0   
181347                     0                      0                  1   
347606                     0                      0                  1   

        premium_age_ratio  premium_vintage_ratio  Policy_Sales_Channel_count  \
347722          -0.692486              -0.417318                   -0.030742   
216616           0.190316              -0.488680                   -1.225801   
136543           1.505898               0.110189                   -1.225801   
320938          -0.411127               0.288878                   -0.144773   
361802           1.489241              -0.370935                    1.122137   
...                   ...                    ...                         ...   
44283           -0.244122              -0.318817                   -1.473353   
139726          -0.333434              -0.415359                   -0.144773   
178713          -1.317921              -0.503141                   -1.597606   
181347          -0.153457               0.141274                   -0.144773   
347606          -0.242582              -0.438327                   -0.144773   

        Region_Code_count  
347722          -0.005717  
216616          -0.139138  
136543           0.090659  
320938          -0.130069  
361802          -0.005717  
...                   ...  
44283            0.073444  
139726           0.090659  
178713           0.142767  
181347          -0.107474  
347606          -0.141136  

[76060 rows x 13 columns]

In [256]:
for column in cat_feat:
    x_train[column] = x_train[column].astype('int')
    x_test[column] = x_test[column].astype('int')

In [257]:
for column in num_feat:
    x_train[column] = x_train[column].astype('int')
    x_test[column] = x_test[column].astype('int')

In [295]:
model_QDA = QDA()
model_QDA.fit(x_train, y_train)

QuadraticDiscriminantAnalysis()

# Quadratic Discriminant Analysis

In [303]:
#Base QDA Without any tuning
QDA_model_default = QDA()
QDA_model_default.fit(x_train, y_train)
y_pred_QDA_default =QDA_model_default.predict(x_test)

#Parameter tuning with GridSearchCV 
#######################
### QDA
#######################
estimator = QuadraticDiscriminantAnalysis()
parameters = {
    'reg_param': (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009),
    'store_covariance': (True, False),
    'tol': (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009), 
                   }

# with GridSearch
grid_search_qda = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'recall',
    n_jobs = -1)

qda_1=grid_search_qda.fit(x_train, y_train)
y_pred_1 =qda_1.predict(x_test)

# qda_2=grid_search_qda.fit(x_train_2, y_train)
# y_pred_2 =qda_2.predict(x_test_2)

# qda_3=grid_search_qda.fit(x_train_3, y_train)
# y_pred_3 =qda_3.predict(x_test_3)

In [305]:
class_report_RF=classification_report(y_test, y_pred_1)
print(class_report_RF)
print('')
print('')
print('QUADRATIC DISCRIMINANT ANALYSIS')
print('Accuracy Score - QDA:', metrics.accuracy_score(y_test, y_pred_1))  
print('Average Precision - QDA:', metrics.average_precision_score(y_test, y_pred_1)) 
print('F1 Score - QDA:', metrics.f1_score(y_test, y_pred_1)) 
print('Precision - QDA:', metrics.precision_score(y_test, y_pred_1)) 
print('Recall - QDA:', metrics.recall_score(y_test, y_pred_1))
print('Weighted Recall & Precision', 0.7*metrics.recall_score(y_test, y_pred_1) + 0.3*metrics.precision_score(y_test, y_pred_1))
print('ROC Score - QDA:', roc_auc_score(y_test, y_pred_1))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82     66797
           1       0.29      0.86      0.44      9263

    accuracy                           0.73     76060
   macro avg       0.63      0.79      0.63     76060
weighted avg       0.89      0.73      0.77     76060



QUADRATIC DISCRIMINANT ANALYSIS
Accuracy Score - QDA: 0.7284380752037865
Average Precision - QDA: 0.26833612073456314
F1 Score - QDA: 0.4360101575512656
Precision - QDA: 0.291812865497076
Recall - QDA: 0.8619237827917521
Weighted Recall & Precision 0.6908905076033492
ROC Score - QDA: 0.7859254376629241


# BaggingClassifier

In [304]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


x_train, x_test, y_train, y_test = train_test_split(training_data,testing_data, test_size=0.2, random_state=42)
bc = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                       random_state=42)
bc.fit(x_train, y_train) 

#BaggingClassifier(...)
y_pred = bc.predict(x_test)
balanced_accuracy_score(y_test, y_pred)

0.5495772290595614

In [306]:
class_report_RF=classification_report(y_test, y_pred)
print(class_report_RF)
print('')
print('')
print('QUADRATIC DISCRIMINANT ANALYSIS')
print('Accuracy Score - QDA:', metrics.accuracy_score(y_test, y_pred))  
print('Average Precision - QDA:', metrics.average_precision_score(y_test, y_pred)) 
print('F1 Score - QDA:', metrics.f1_score(y_test, y_pred)) 
print('Precision - QDA:', metrics.precision_score(y_test, y_pred)) 
print('Recall - QDA:', metrics.recall_score(y_test, y_pred))
print('Weighted Recall & Precision', 0.7*metrics.recall_score(y_test, y_pred) + 0.3*metrics.precision_score(y_test, y_pred))
print('ROC Score - QDA:', roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.96      0.92     66797
           1       0.34      0.14      0.19      9263

    accuracy                           0.86     76060
   macro avg       0.61      0.55      0.56     76060
weighted avg       0.82      0.86      0.84     76060



QUADRATIC DISCRIMINANT ANALYSIS
Accuracy Score - QDA: 0.8627267946358138
Average Precision - QDA: 0.15142529306047325
F1 Score - QDA: 0.19393190766617768
Precision - QDA: 0.3403794037940379
Recall - QDA: 0.13559322033898305
Weighted Recall & Precision 0.1970290753754995
ROC Score - QDA: 0.5495772290595614


# BalancedBaggingClassifier

In [312]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.ensemble import BalancedBaggingClassifier

# Original dataset shape Counter({1: 900, 0: 100})
x_train, x_test, y_train, y_test = train_test_split(training_data,testing_data, test_size=0.2, random_state=42)

bbc = BalancedBaggingClassifier(random_state=42)
bbc.fit(x_train, y_train)
# BalancedBaggingClassifier(...)
y_pred = bbc.predict(x_test)
print(confusion_matrix(y_test, y_pred))

[[51972 14825]
 [ 3002  6261]]


In [313]:
class_report_RF=classification_report(y_test, y_pred)
print(class_report_RF)
print('')
print('')
print('QUADRATIC DISCRIMINANT ANALYSIS')
print('Accuracy Score - QDA:', metrics.accuracy_score(y_test, y_pred))  
print('Average Precision - QDA:', metrics.average_precision_score(y_test, y_pred)) 
print('F1 Score - QDA:', metrics.f1_score(y_test, y_pred)) 
print('Precision - QDA:', metrics.precision_score(y_test, y_pred)) 
print('Recall - QDA:', metrics.recall_score(y_test, y_pred))
print('Weighted Recall & Precision', 0.7*metrics.recall_score(y_test, y_pred) + 0.3*metrics.precision_score(y_test, y_pred))
print('ROC Score - QDA:', roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.78      0.85     66797
           1       0.30      0.68      0.41      9263

    accuracy                           0.77     76060
   macro avg       0.62      0.73      0.63     76060
weighted avg       0.87      0.77      0.80     76060



QUADRATIC DISCRIMINANT ANALYSIS
Accuracy Score - QDA: 0.7656192479621352
Average Precision - QDA: 0.24016614566446143
F1 Score - QDA: 0.4126000856700386
Precision - QDA: 0.2969268709096083
Recall - QDA: 0.6759149303681313
Weighted Recall & Precision 0.5622185125305743
ROC Score - QDA: 0.7269869126143395


# BalancedRandomForestClassifier

In [309]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(x_train, y_train)
# BalancedRandomForestClassifier(...)
y_pred = brf.predict(x_test)
balanced_accuracy_score(y_test, y_pred)

0.7943267546406454

In [310]:
class_report_RF=classification_report(y_test, y_pred)
print(class_report_RF)
print('')
print('')
print('QUADRATIC DISCRIMINANT ANALYSIS')
print('Accuracy Score - QDA:', metrics.accuracy_score(y_test, y_pred))  
print('Average Precision - QDA:', metrics.average_precision_score(y_test, y_pred)) 
print('F1 Score - QDA:', metrics.f1_score(y_test, y_pred)) 
print('Precision - QDA:', metrics.precision_score(y_test, y_pred)) 
print('Recall - QDA:', metrics.recall_score(y_test, y_pred))
print('Weighted Recall & Precision', 0.7*metrics.recall_score(y_test, y_pred) + 0.3*metrics.precision_score(y_test, y_pred))
print('ROC Score - QDA:', roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.68      0.81     66797
           1       0.28      0.91      0.43      9263

    accuracy                           0.71     76060
   macro avg       0.63      0.79      0.62     76060
weighted avg       0.90      0.71      0.76     76060



QUADRATIC DISCRIMINANT ANALYSIS
Accuracy Score - QDA: 0.7097948987641336
Average Precision - QDA: 0.26838901352494765
F1 Score - QDA: 0.43197200133817126
Precision - QDA: 0.2835856196783349
Recall - QDA: 0.9060779445104178
Weighted Recall & Precision 0.7193302470607928
ROC Score - QDA: 0.7943267546406453


# RUSBoostClassifier

In [314]:
from imblearn.ensemble import RUSBoostClassifier
rusboost = RUSBoostClassifier(n_estimators=200, algorithm='SAMME.R',
                              random_state=42)
rusboost.fit(x_train, y_train)
y_pred = rusboost.predict(x_test)
balanced_accuracy_score(y_test, y_pred)

0.7975698955696962

In [316]:
class_report_RF=classification_report(y_test, y_pred)
print(class_report_RF)
print('')
print('')
print('QUADRATIC DISCRIMINANT ANALYSIS')
print('Accuracy Score - QDA:', metrics.accuracy_score(y_test, y_pred))  
print('Average Precision - QDA:', metrics.average_precision_score(y_test, y_pred)) 
print('F1 Score - QDA:', metrics.f1_score(y_test, y_pred)) 
print('Precision - QDA:', metrics.precision_score(y_test, y_pred)) 
print('Recall - QDA:', metrics.recall_score(y_test, y_pred))
print('Weighted Recall & Precision', 0.7*metrics.recall_score(y_test, y_pred) + 0.3*metrics.precision_score(y_test, y_pred))
print('ROC Score - QDA:', roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.67      0.80     66797
           1       0.28      0.92      0.43      9263

    accuracy                           0.70     76060
   macro avg       0.63      0.80      0.61     76060
weighted avg       0.90      0.70      0.75     76060



QUADRATIC DISCRIMINANT ANALYSIS
Accuracy Score - QDA: 0.7018537996318696
Average Precision - QDA: 0.2683097437193024
F1 Score - QDA: 0.43018317963665603
Precision - QDA: 0.2803432239470754
Recall - QDA: 0.9241066609089927
Weighted Recall & Precision 0.7309776298204175
ROC Score - QDA: 0.7975698955696962


# EasyEnsembleClassifier

In [319]:
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(x_train, y_train)


y_pred = eec.predict(x_test)
balanced_accuracy_score(y_test, y_pred)


0.7963268512530204

In [320]:
class_report_RF=classification_report(y_test, y_pred)
print(class_report_RF)
print('')
print('')
print('QUADRATIC DISCRIMINANT ANALYSIS')
print('Accuracy Score - QDA:', metrics.accuracy_score(y_test, y_pred))  
print('Average Precision - QDA:', metrics.average_precision_score(y_test, y_pred)) 
print('F1 Score - QDA:', metrics.f1_score(y_test, y_pred)) 
print('Precision - QDA:', metrics.precision_score(y_test, y_pred)) 
print('Recall - QDA:', metrics.recall_score(y_test, y_pred))
print('Weighted Recall & Precision', 0.7*metrics.recall_score(y_test, y_pred) + 0.3*metrics.precision_score(y_test, y_pred))
print('ROC Score - QDA:', roc_auc_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.66      0.79     66797
           1       0.28      0.93      0.43      9263

    accuracy                           0.69     76060
   macro avg       0.63      0.80      0.61     76060
weighted avg       0.90      0.69      0.75     76060



QUADRATIC DISCRIMINANT ANALYSIS
Accuracy Score - QDA: 0.6942808309229556
Average Precision - QDA: 0.26549056007138017
F1 Score - QDA: 0.4259227255894334
Precision - QDA: 0.2761026822866654
Recall - QDA: 0.9312317823599265
Weighted Recall & Precision 0.7346930523379481
ROC Score - QDA: 0.7963268512530205
